# TRANSFORMER para la Traducción de Texto

> Basado en:https://www.udemy.com/course/procesamiento-del-lenguaje-natural/learn/lecture/21502260#overview

- Author: Juan Gabriel Gomila
- Course: "Procesamiento del Lenguaje Natural Moderno en Python.

## Importar las dependencias

**Paper original**: All you need is Attention https://arxiv.org/pdf/1706.03762.pdf

In [1]:
import os
import numpy as np
import pandas as pd
import re
import time

from joblib import dump, load

In [2]:
try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
from mlearner.nlp import Transformer
from mlearner.nlp import Processor_data

In [4]:
%load_ext autoreload
%autoreload 2

%matplotlib inline


Bad key savefig.frameon in file C:\Users\AUTIS\Anaconda3\envs\Tensorflow 2.0\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\AUTIS\Anaconda3\envs\Tensorflow 2.0\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.0/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\AUTIS\Anaconda3\envs\Tensorflow 2.0\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.s

In [5]:
TRAIN = True

## Pre Procesado de Datos



## Carga de Ficheros

In [6]:
with open("data/europarl-v7.es-en.en", 
          mode = "r", encoding = "utf-8") as f:
    europarl_en = f.read()
with open("data/europarl-v7.es-en.es", 
          mode = "r", encoding = "utf-8") as f:
    europarl_es = f.read()
with open("data/P85-Non-Breaking-Prefix.en", 
          mode = "r", encoding = "utf-8") as f:
    non_breaking_prefix_en = f.read()
with open("data/P85-Non-Breaking-Prefix.en", 
          mode = "r", encoding = "utf-8") as f:
    non_breaking_prefix_es = f.read()

In [7]:
europarl_en[:230]

'Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive peri'

In [8]:
europarl_es[:225]

'Reanudación del período de sesiones\nDeclaro reanudado el período de sesiones del Parlamento Europeo, interrumpido el viernes 17 de diciembre pasado, y reitero a Sus Señorías mi deseo de que hayan tenido unas buenas vacaciones'

## Limpieza de datos

Definimos funcion de procesado de texto basada en expresiones regulares

In [9]:
def Function_clean(text):
    
    # Eliminamos la @ y su mención
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    # Eliminamos los links de las URLs
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    return text


Se procesan los textos para cada uno de los idiomas:

In [10]:
processor_en = Processor_data(target_vocab_size=2**13, 
                              language="en", 
                              function = Function_clean,
                              name="processor_en",
                             )
processor_es = Processor_data(target_vocab_size=2**13, 
                              language="es", 
                              function = Function_clean,
                              name="processor_es"
                              )

In [11]:
if not os.path.isfile("data/corpus_en.csv"):
    corpus_en = europarl_en
    corpus_en = processor_en.clean(corpus_en)
    pd.DataFrame(corpus_en).to_csv("data/corpus_en.csv", index=False)

if not os.path.isfile("data/corpus_es.csv"):
    corpus_es = europarl_es
    corpus_es = processor_es.clean(corpus_es)
    pd.DataFrame(corpus_es).to_csv("data/corpus_es.csv", index=False)

corpus_en = pd.read_csv("data/corpus_en.csv")    
corpus_es = pd.read_csv("data/corpus_es.csv")

Exploramos los textos para cada idioma:

In [12]:
corpus_en[0:2]

,0
0,Resumption of the session
1,I declare resumed the session of the European ...


In [13]:
corpus_es[0:2]

,0
0,Reanudación del período de sesiones
1,Declaro reanudado el período de sesiones del P...


In [14]:
len(corpus_en), len(corpus_es)

(1965735, 1965735)

## Tokenizar el Texto

Tokenizado del texto sin aplicar limpieza (aplicada en el apartado anterior) y sin padding:

In [15]:
if not os.path.isfile('data/processor_en.joblib'):
    tokens_en = processor_en.process_text(corpus_en, 
                                             isclean=True, 
                                             padding=False)
    dump(processor_en, 'data/processor_en.joblib')
else:
    processor_en = load('data/processor_en.joblib')

In [16]:
if not os.path.isfile('data/processor_es.joblib'):
    tokens_es = processor_es.process_text(corpus_es, 
                                             isclean=True, 
                                             padding=False)
    dump(processor_es, 'data/processor_es.joblib')
else:
    processor_es = load('data/processor_es.joblib')

Tamaño de Vocabulario para los dos idiomas.

In [17]:
if not os.path.isfile("data/inputs.csv") and not os.path.isfile("data/outputs.csv"):
    VOCAB_SIZE_EN = processor_en.tokenizer.vocab_size + 2
    VOCAB_SIZE_ES = processor_es.tokenizer.vocab_size + 2

    print(VOCAB_SIZE_EN, VOCAB_SIZE_ES)

Sustituimos los valores NaN con valores vacios:

In [18]:
if not os.path.isfile("data/inputs.csv") and not os.path.isfile("data/outputs.csv"):
    corpus_es = corpus_es.fillna(" ")
    corpus_en = corpus_en.fillna(" ")

Preparación de las frases como inputs/outputs del Modelo:

> _**[ \INICIO ]**_ + frase + _**[ \FIN ]**_

- **[ \INICIO ]**: Carácter que determina el inicio de frase.
- **[ \FIN ]**: Carácter que determina el final de frase.

In [19]:
if not os.path.isfile("data/inputs.csv") and not os.path.isfile("data/outputs.csv"):
    inputs = [[VOCAB_SIZE_EN-2] + \
              processor_en.tokenizer.encode(sentence[0]) + [VOCAB_SIZE_EN-1] \
                for sentence in corpus_en.values]

    outputs = [[VOCAB_SIZE_ES-2] + \
               processor_es.tokenizer.encode(sentence[0]) + [VOCAB_SIZE_ES-1] 
                for sentence in corpus_es.values ]
    len(inputs), len(outputs)

## Eliminamos las frases demasiado largas

In [20]:
MAX_LENGTH = 20

if not os.path.isfile("data/inputs.csv") and not os.path.isfile("data/outputs.csv"):
    idx_to_remove = [count for count, sent in enumerate(inputs)
                     if len(sent) > MAX_LENGTH]
    if len(idx_to_remove) > 0:
        for idx in reversed(idx_to_remove):
            del inputs[idx]
            del outputs[idx]

    idx_to_remove = [count for count, sent in enumerate(outputs)
                     if len(sent) > MAX_LENGTH]
    if len(idx_to_remove) > 0:
        for idx in reversed(idx_to_remove):
            del inputs[idx]
            del outputs[idx]

    pd.DataFrame(inputs).to_csv("data/inputs.csv", index=False)
    pd.DataFrame(outputs).to_csv("data/outputs.csv", index=False)

## Creamos las entradas y las salidas

A medida que entrenamos con bloques, necesitaremos que cada entrada tenga la misma longitud. Rellenamos con el token apropiado, y nos aseguraremos de que este token de relleno no interfiera con nuestro entrenamiento más adelante.

In [21]:
inputs = pd.read_csv("data/inputs.csv").fillna(0).astype(int)   
outputs = pd.read_csv("data/outputs.csv").fillna(0).astype(int)

len(inputs), len(outputs)

(411131, 411131)

In [22]:
MAX_LENGTH = 20

VOCAB_SIZE_EN = 8198
VOCAB_SIZE_ES = 8225 

inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs.values,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs.values,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=MAX_LENGTH)

Se crea el daset generador para servir los inputs/outputs procesados.

In [23]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

## Modelo Transformer - Entrenamiento

In [24]:
tf.keras.backend.clear_session()

# Hiper Parámetros
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT_RATE = 0.1 # 0.1

model_Transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_ES,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout_rate=DROPOUT_RATE)

Bucle de entrenamiento

In [25]:
from mlearner.nlp import Transformer_train

In [ ]:
EPOCHS = 1

Transformer_train(model_Transformer,
                  dataset,
                  d_model=D_MODEL,
                  train=TRAIN,
                  epochs=EPOCHS,
                  checkpoint_path="ckpt/",
                  max_to_keep=5)

The last checkpoint has been restored
Inicio del epoch 1
Epoch 1 Lote 0 Pérdida 1.5254 Precisión 0.4120
Epoch 1 Lote 50 Pérdida 1.2794 Precisión 0.4312
Epoch 1 Lote 100 Pérdida 1.2828 Precisión 0.4283
Epoch 1 Lote 150 Pérdida 1.2888 Precisión 0.4290
Epoch 1 Lote 200 Pérdida 1.2800 Precisión 0.4284
Epoch 1 Lote 250 Pérdida 1.2772 Precisión 0.4293
Epoch 1 Lote 300 Pérdida 1.2802 Precisión 0.4306
Epoch 1 Lote 350 Pérdida 1.2769 Precisión 0.4307
Epoch 1 Lote 400 Pérdida 1.2755 Precisión 0.4304
Epoch 1 Lote 450 Pérdida 1.2708 Precisión 0.4305
Epoch 1 Lote 500 Pérdida 1.2662 Precisión 0.4306
Epoch 1 Lote 550 Pérdida 1.2648 Precisión 0.4307
Epoch 1 Lote 600 Pérdida 1.2632 Precisión 0.4307
Epoch 1 Lote 650 Pérdida 1.2624 Precisión 0.4299
Epoch 1 Lote 700 Pérdida 1.2614 Precisión 0.4303
Epoch 1 Lote 750 Pérdida 1.2577 Precisión 0.4314
Epoch 1 Lote 800 Pérdida 1.2538 Precisión 0.4324
Epoch 1 Lote 850 Pérdida 1.2489 Precisión 0.4335
Epoch 1 Lote 900 Pérdida 1.2450 Precisión 0.4343
Epoch 1 Lote 95

## Evaluación

In [ ]:
def evaluate(inp_sentence):
    inp_sentence = \
        [VOCAB_SIZE_EN-2] + processor_en.tokenizer.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    enc_input = tf.expand_dims(inp_sentence, axis=0)
    
    output = tf.expand_dims([VOCAB_SIZE_ES-2], axis=0)
    
    for _ in range(MAX_LENGTH):
        predictions = model_Transformer(enc_input, output, False) #(1, seq_length, VOCAB_SIZE_ES)
        
        prediction = predictions[:, -1:, :]
        
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)
        
        if predicted_id == VOCAB_SIZE_ES-1:
            return tf.squeeze(output, axis=0)
        
        output = tf.concat([output, predicted_id], axis=-1)
        
    return tf.squeeze(output, axis=0)

In [ ]:
def translate(sentence):
    output = evaluate(sentence).numpy()
    
    predicted_sentence = processor_es.tokenizer.decode(
        [i for i in output if i < VOCAB_SIZE_ES-2]
    )
    
    print("Entrada: {}".format(sentence))
    print("Traducción predicha: {}".format(predicted_sentence))

## Predicciones

In [ ]:
translate("I have got a house")

In [ ]:
translate("This is a problem we have to solve.")

In [ ]:
translate("This is a really powerful tool!")

In [ ]:
translate("This is an interesting course about Natural Language Processing")